## Classification ML project

In [ ]:
from IPython.core.display import SVG
SVG(filename='pipeline.svg')

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
import logistic as log
import split as spl
import least_squares as lst
import helpers as hlp
import pre_processing as pre

In [ ]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
DATA_TEST_PATH = '../data/test.csv'

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import pandas as pd # cannot use external libraries, just pandas for data exploration

In [ ]:
y, tX, ids = hlp.load_csv_data(DATA_TRAIN_PATH)
_, tX_test, ids_test = hlp.load_csv_data(DATA_TEST_PATH)

In [ ]:
print(tX.shape)
print(y.shape)

## Data exploration

In [ ]:
data = pd.read_csv(DATA_TRAIN_PATH)
test_data = pd.read_csv(DATA_TEST_PATH)
dic = {'s':1,'b':-1}
data.Prediction = data.Prediction.map(dic)
test_data.Prediction = test_data.Prediction.map(dic)
data.head(10)

In [ ]:
mask = data.isin([-999]).any(axis = 1)
print(len(data[mask]))
print(len(data))

_The vast majoriy of our data has -999 values: we'd better handle it carefully_

In [ ]:
#data.replace(to_replace = -999,value = np.nan, inplace = True)

In [ ]:
#replace_val = np.nan
#tX = np.where(tX == -999,replace_val,tX)

In [ ]:
std = np.nanstd(tX,axis = 0)
mean = np.nanmean(tX,axis = 0)

In [ ]:
print('Train set size: {} samples x {} features'.format(pd.DataFrame(tX).shape[0], pd.DataFrame(tX).shape[1]))
print('Test set size: {} samples x {} features'.format(test_data.shape[0], pd.DataFrame(tX).shape[1]))

In [ ]:
data.info()

In [ ]:
data.describe()

## Data Cleaning

In [ ]:
#totrash before submit: we use pandas to know to which index PRI_jet_num does correspond.
np.where(data.columns.values == "PRI_jet_num")

In [ ]:
x_trains = spl.split_categorical_data(tX,24,labels = y,split = False)
x_test = spl.split_categorical_data(tX_test,24,split = False)

In [ ]:
#deso pour cette boucle a la c++ de mords moi le noeud, dois y avoir un moyen de faire ca clean
for i in range(len(x_trains)):
    pre.clean_variance(x_trains[i][0],x_test[i],inplace = True)
    pre.clean_value(x_trains[i][0],-999,np.nan,inplace = True)
    pre.clean_value(x_test[i][0],-999,np.nan,inplace = True)
    x_std,x_mean,_ =  pre.standardize_data(x_trains[i][0], inplace = True)
    pre.standardize_data(x_test[i], x_mean,x_std,inplace = True)
    pre.clean_value(x_trains[i][0],np.nan,0,inplace = True)
    pre.clean_value(x_test[i][0],np.nan,0,inplace = True)
    #todo: pca, feature exp

TODO:At this point todo: in pre_processing.py, all the functions used here basically. especially change the form of standardize data so that it does not make the job of clean_value.


## Data Processing:

In [ ]:
#trash random dataframe
np.random.seed(2)
df = pd.DataFrame(np.random.randint(-1002,-995,size =(3,4)), columns=list('ABCD'))
df.replace(to_replace = -999,value = np.nan)

## Put your useful trash here

In [ ]:
#Yann
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)